In [ ]:
!pip install -q langchain_groq
!pip install -q python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.4/415.4 kB 24.8 MB/s eta 0:00:00


In [ ]:
!pip install -q docx2txt pypdf unstructured
!pip install -q langchain_community
!pip install -q sentence_transformers
!pip install -q langchain_chroma

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 44.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.8/166.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
%%writefile .env
GROQ_API_KEY = "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"

Writing .env


In [ ]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
#from langchain_openai import OpenAIEmbeddings
from typing import List
from langchain_core.documents import Document
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough


In [ ]:
load_dotenv()
llm = ChatGroq(temperature=0.7)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=200,
    #separators=["\n\n", "\n", " ", ""]
    length_function=len
)

docx_loader = Docx2txtLoader("/content/docs/rag_doc.docx")
documents = docx_loader.load()
#print(len(documents))

splits = text_splitter.split_documents(documents)

In [ ]:
#for i, split in enumerate(splits):
#  print(i)
#  print(len(str((split))))
#  print(split)

In [ ]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
document_embeddings = embedding_function.embed_documents([split.page_content for split in splits])

In [ ]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
collection_name = 'doc_collection_1'
vectorStore = Chroma.from_documents(
    documents=splits,
    embedding=embedding_function,
    collection_name=collection_name
)

In [ ]:
#query = "What event caused the people of Eldoria to disappear?"
#search_results = vectorStore.max_marginal_relevance_search(query, k=2, lambda_mult=0.5)

#for i, result in enumerate(search_results):
#  print(i)
#  print(result.page_content)

In [ ]:
retriever = vectorStore.as_retriever(search_kwargs={"k": 2}, lambda_mult = 0.5)

In [ ]:
#retriever.invoke(query)

In [ ]:
template = """
Answer the question based only on the following context in 20 words:
{context}

Question: {question}

Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
def doc2str(docs):
  return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
rag_chain = (
    {"context": retriever | doc2str, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
query = "Who led the archaeological team that discovered Eldoria?"
question = query
response = rag_chain.invoke(question)
print(response)

Dr. Amelia Carter led the archaeological team that discovered Eldoria.


In [ ]:
!pip install gtts

In [105]:
from gtts import gTTS
import IPython.display as ipd

while True:
    question = input("\nAsk a question (or type 'exit' to quit): ")

    if question.lower() == "exit":
        print("Exiting...")
        break  # Stop the loop

    response = rag_chain.invoke(question)
    print("\nAnswer:", response)

    # Convert text response to speech
    tts = gTTS(text=response, lang='en')
    tts.save("response.mp3")  # Save as MP3

    # Play the audio in Colab
    ipd.display(ipd.Audio("response.mp3"))


Ask a question (or type 'exit' to quit): Why did the portal collapse

Answer: The portal collapsed because the crystal powering it was depleted of energy after the team touched the edge of the portal, possibly due to the energy surge from the potential connection to Eldoria.


KeyboardInterrupt: Interrupted by user

In [ ]:
while True:
    question = input("\nAsk a question (or type 'exit' to quit): ")

    if question.lower() == "exit":
        print("Exiting...")
        break  # Stop the loop

    response = rag_chain.invoke(question)
    print("\nAnswer:", response)


Ask a question (or type 'exit' to quit): Who led the archaeological team that discovered Eldoria

Answer: Dr. Amelia Carter led the archaeological team that discovered Eldoria.

Ask a question (or type 'exit' to quit): Where was Eldoria located?

Answer: The text does not provide information on Eldoria's original location in this world.

Ask a question (or type 'exit' to quit): What did the team find inside the temple

Answer: The team found a chamber with golden artifacts, intricate murals depicting an ancient civilization, and a central pedestal holding a deep azure crystal. One mural showed the people of Eldoria vanishing in a blinding light.

Ask a question (or type 'exit' to quit): What event caused the people of Eldoria to disappear?

Answer: The inscriptions suggest a event called "The Great Departure" caused Eldorians to leave their world for another realm.

Ask a question (or type 'exit' to quit): What was the significance of the murals inside the temple

Answer: The murals d